In [1]:
# Imports.
import pygame
import moderngl
import glm
import numpy as np
from LoadObject import getObjectData

pygame 2.6.0 (SDL 2.28.4, Python 3.12.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# Initialize Pygame & OpenGL.
def init_window(width=800, height=600):
    pygame.init()
    pygame.display.set_mode((width, height), pygame.OPENGL | pygame.DOUBLEBUF)
    pygame.display.set_caption("Assignment 07: Colin Kirby")

    # Create and Set up OpenGL.
    ctx = moderngl.create_context()
    
    ctx.enable(moderngl.CULL_FACE)
    ctx.enable(moderngl.DEPTH_TEST)

    return ctx

In [3]:
# Initialize Window.
ctx = init_window()

In [4]:
# Load the Vertex Data for our Platform, Teapot, and Light Source.
vertex_data_platform, _ = getObjectData("cube.obj", normal=True)
vertex_data_teapot, _ = getObjectData("teapot_with_texCoords.obj", normal=True)
vertex_data_light, _ = getObjectData("20_icosahedron.obj", normal=True)  # Light source object

Normal exists
Normal exists
Normal computed.


In [5]:
# Global VAO.
vao_platform = None
vao_teapot = None
vao_light = None
program_platform = None
program_teapot = None
program_light = None

In [6]:
# Upload the Vertex Data for Platform. 
def set_vert_data_platform():
    global vao_platform, program_platform

    # Convert Vertex Data to NP Array.
    vertex_data_platform_np = np.array(vertex_data_platform, dtype='f4')
    
    # Create a buffer on GPU & Load Vertex Data.
    vbo_platform = ctx.buffer(vertex_data_platform_np.tobytes())

    # Compile Shader Program.
    program_platform = ctx.program(
        # Vertex Shader.
        vertex_shader=""" 
        #version 330
        in vec3 in_position;
        in vec3 in_normal;

        uniform mat4 model;
        uniform mat4 view;
        uniform mat4 proj;
        uniform mat3 normalMatrix;

        out vec3 frag_pos;
        out vec3 frag_normal;

        void main() {
            frag_pos = vec3(model * vec4(in_position, 1.0));
            frag_normal = normalize(normalMatrix * in_normal);
            gl_Position = proj * view * vec4(frag_pos, 1.0);
        }
        """,
        # Updated Fragment Shader.
        fragment_shader="""
        #version 330
        in vec3 frag_pos;
        in vec3 frag_normal;

        uniform vec3 light_pos;    // For point light: light position; for directional light: light direction
        uniform vec3 light_color;
        uniform vec3 view_pos;
        uniform vec3 object_color;
        uniform bool is_point_light; // Boolean to differentiate between point and directional lights

        out vec4 frag_color;

        void main() {
            // Ambient lighting
            vec3 ambient = 0.1 * object_color;

            // Diffuse lighting
            vec3 norm = normalize(frag_normal);
            vec3 light_dir;
            
            // If point light, calculate direction from fragm dent to light position
            if (is_point_light) {
                light_dir = normalize(light_pos - frag_pos);
            } else {
                // If directional light, use the normalized light direction directly
                light_dir = normalize(light_pos);
            }
            
            float diff = max(dot(norm, light_dir), 0.0);
            vec3 diffuse = diff * light_color * object_color;

            // Final color
            vec3 result = ambient + diffuse;
            frag_color = vec4(result, 1.0);
        }
        """
    )

    # Create the VAO for the Platform.
    vao_platform = ctx.vertex_array(
        program_platform,
        [
            (vbo_platform, '3f 3f', 'in_position', 'in_normal')
        ]
    )


In [7]:
# Upload Vertex Data for Teapot.
def set_vert_data_teapot():
    global vao_teapot, program_teapot

    # Convert Teapot Vertex Data & Convert to NP Array.
    vertex_data_teapot_np = np.array(vertex_data_teapot, dtype='f4')
    
    # Create buffer on GPU & load Teapot Vertex Data.
    vbo_teapot = ctx.buffer(vertex_data_teapot_np.tobytes())

    # Compile Shader Program.
    program_teapot = ctx.program(
        # Vertex Shader.
        vertex_shader="""
        #version 330
        in vec3 in_position;
        in vec3 in_normal;

        uniform mat4 model;
        uniform mat4 view;
        uniform mat4 proj;
        uniform mat3 normalMatrix;

        out vec3 frag_pos;
        out vec3 frag_normal;

        void main() {
            frag_pos = vec3(model * vec4(in_position, 1.0));
            frag_normal = normalize(normalMatrix * in_normal);
            gl_Position = proj * view * vec4(frag_pos, 1.0);
        }
        """,
        # Updated Fragment Shader.
        fragment_shader="""
        #version 330
        in vec3 frag_pos;
        in vec3 frag_normal;

        uniform vec3 light_pos;    // For point light: light position; for directional light: light direction
        uniform vec3 light_color;
        uniform vec3 view_pos;
        uniform vec3 object_color;
        uniform bool is_point_light; // Boolean to differentiate between point and directional lights

        out vec4 frag_color;

        void main() {
            // Ambient lighting
            vec3 ambient = 0.1 * object_color;

            // Diffuse lighting
            vec3 norm = normalize(frag_normal);
            vec3 light_dir;
            
            // If point light, calculate direction from fragment to light position
            if (is_point_light) {
                light_dir = normalize(light_pos - frag_pos);
            } else {
                // If directional light, use the normalized light direction directly
                light_dir = normalize(light_pos);
            }
            
            float diff = max(dot(norm, light_dir), 0.0);
            vec3 diffuse = diff * light_color * object_color;

            // Final color
            vec3 result = ambient + diffuse;
            frag_color = vec4(result, 1.0);
        }
        """
    )

    # Create VAO for Teapot.
    vao_teapot = ctx.vertex_array(
        program_teapot,
        [
            (vbo_teapot, '3f 3f', 'in_position', 'in_normal')
        ]
    )


In [8]:
# Upload Vertex Data for Light Source.
def set_vert_data_light():
    global vao_light, program_light

    # Convert Light Vertex Data & Convert to NP Array.
    vertex_data_light_np = np.array(vertex_data_light, dtype='f4')
    
    # Create buffer on GPU & load Light Source Vertex Data.
    vbo_light = ctx.buffer(vertex_data_light_np.tobytes())

    # Compile Light Shader Program (Simple white color).
    program_light = ctx.program(
        # Vertex Shader.
        vertex_shader="""
        #version 330
        in vec3 in_position;

        uniform mat4 model;
        uniform mat4 view;
        uniform mat4 proj;

        void main() {
            gl_Position = proj * view * model * vec4(in_position, 1.0);
        }
        """,
        # Fragment Shader.
        fragment_shader="""
        #version 330
        out vec4 frag_color;

        void main() {
            frag_color = vec4(1.0, 1.0, 1.0, 1.0);  // Render the light source as white
        }
        """
    )

    # Create VAO for Light Source.
    vao_light = ctx.vertex_array(
        program_light,
        [
            (vbo_light, '3f', 'in_position')
        ]
    )

In [9]:
# Initialize Vertex Data.
set_vert_data_platform()
set_vert_data_teapot()
set_vert_data_light()

In [10]:
# Projection Matrix
fov = 60.0  
width, height = 800, 600
aspect_ratio = width / height  
near_plane = 10.0  
far_plane = 45.0 
proj_matrix = glm.perspective(glm.radians(fov), aspect_ratio, near_plane, far_plane)

In [11]:
# Static Camera Position (based on Assignment 06 instructions).
view_matrix = glm.lookAt(
    glm.vec3(20.0, 10.0, 0.0),  # Camera position
    glm.vec3(0.0, 0.0, 0.0),    # Look at the origin
    glm.vec3(0.0, 1.0, 0.0)     # Up vector
)

In [ ]:
# Rendering Loop.
running = True
is_point_light = True  # Toggle between point light and directional light

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        if event.type == pygame.KEYDOWN and event.key == pygame.K_l:
            is_point_light = not is_point_light  # Toggle the light mode

    # Time and light position for orbiting light.
    time_in_seconds = pygame.time.get_ticks() / 1000.0
    light_pos = glm.vec3(15.0 * glm.cos(time_in_seconds), 15.0, 15.0 * glm.sin(time_in_seconds))
    
    # Toggle between point and directional light.
    if is_point_light:
        light_direction_or_pos = light_pos  # Use position for point light
    else:
        light_direction_or_pos = glm.normalize(light_pos)  # Use normalized direction for directional light

    # Render Platform.
    if vao_platform is not None and program_platform is not None:
        try:
            # Platform matrix (scaling and positioning)
            model_matrix_platform = glm.scale(glm.mat4(1.0), glm.vec3(10.0, 0.05, 10.0))
            normal_matrix_platform = glm.transpose(glm.inverse(glm.mat3(model_matrix_platform)))

            # Write matrices and uniforms.
            program_platform['model'].write(np.array(model_matrix_platform, dtype='f4').T.flatten().tobytes())
            program_platform['normalMatrix'].write(np.array(normal_matrix_platform, dtype='f4').T.flatten().tobytes())
            program_platform['view'].write(np.array(view_matrix, dtype='f4').T.flatten().tobytes())
            program_platform['proj'].write(np.array(proj_matrix, dtype='f4').T.flatten().tobytes())
            program_platform['light_pos'].write(np.array(light_direction_or_pos, dtype='f4').T.flatten().tobytes())
            program_platform['light_color'].write(np.array([1.0, 1.0, 1.0], dtype='f4').tobytes())  # White light
            program_platform['object_color'].write(np.array([0.0, 0.6, 0.0], dtype='f4').tobytes())  # Floor color

            # Pass Boolean Flag for Light Type.
            program_platform['is_point_light'].value = is_point_light

            # Clear and render.
            ctx.clear(0.1, 0.1, 0.1)
            vao_platform.render(moderngl.TRIANGLES)

        except Exception as e:
            pass 

    # Render Teapot.
    if vao_teapot is not None and program_teapot is not None:
        try:
            # Rotate Teapot 90 Degrees like in PowerPoint Slides.
            rotation_matrix_teapot = glm.rotate(glm.mat4(1.0), glm.radians(90.0), glm.vec3(0.0, 1.0, 0.0))

            # Apply Scaling to Teapot, then Translate it.
            model_matrix_teapot = glm.scale(rotation_matrix_teapot, glm.vec3(0.3))
            model_matrix_teapot = glm.translate(model_matrix_teapot, glm.vec3(0.0, 7.875, 0.0))
    
            # Compute the normal matrix for lighting
            normal_matrix_teapot = glm.transpose(glm.inverse(glm.mat3(model_matrix_teapot)))

            # Write matrices and uniforms.
            program_teapot['model'].write(np.array(model_matrix_teapot, dtype='f4').T.flatten().tobytes())
            program_teapot['normalMatrix'].write(np.array(normal_matrix_teapot, dtype='f4').T.flatten().tobytes())
            program_teapot['view'].write(np.array(view_matrix, dtype='f4').T.flatten().tobytes())
            program_teapot['proj'].write(np.array(proj_matrix, dtype='f4').T.flatten().tobytes())
            program_teapot['light_pos'].write(np.array(light_direction_or_pos, dtype='f4').T.flatten().tobytes())
            program_teapot['light_color'].write(np.array([1.0, 1.0, 1.0], dtype='f4').tobytes())  # White light
            program_teapot['object_color'].write(np.array([1.0, 0.7, 0.5], dtype='f4').tobytes())  # Teapot color

            # Pass Boolean Flag for Light Type.
            program_teapot['is_point_light'].value = is_point_light

            # Render Teapot.
            vao_teapot.render(moderngl.TRIANGLES)

        except Exception as e:
            pass

    # Render Light Source.
    if vao_light is not None and program_light is not None:
        try:
            # Light source model matrix (translation and scaling)
            model_matrix_light = glm.translate(glm.mat4(1.0), light_pos)
            model_matrix_light = glm.scale(model_matrix_light, glm.vec3(0.1))  # Scale by 0.1

            # Write matrices for the light source.
            program_light['model'].write(np.array(model_matrix_light, dtype='f4').T.flatten().tobytes())
            program_light['view'].write(np.array(view_matrix, dtype='f4').T.flatten().tobytes())
            program_light['proj'].write(np.array(proj_matrix, dtype='f4').T.flatten().tobytes())

            # Render the light source object.
            vao_light.render(moderngl.TRIANGLES)

        except Exception as e:
            pass

    # Swap Display Buffers.
    pygame.display.flip()

# Clean Up.
pygame.quit()
